In [ ]:
# USER: create the starting maps for each scenario

In [ ]:
import arcpy
# Get the project
aprx = arcpy.mp.ArcGISProject('CURRENT')

In [ ]:
# common vars (change according to project file)
lyr_to_change = ['FC_Result_COC']  # this is the only layer that changes

prefix = 'CWK'
suffix = ''

In [ ]:
# these are the maps that the others will be modeled after (USER creates these)
# NOTE: don't forget to create the Sediment map template for Soil/Sediment (ug/kg)
# also, you don't need to create "PFOS GW", just "PFOA GW"

list_of_start_maps = ['CWK_PFOA_Nearsite_GW', 'CWK_PFNA_Nearsite_GW', 'CWK_PFNA_Nearsite_GW', \
                      'CWK_PFNA_Nearsite_Soil']

In [ ]:
# Loops through and runs once for each of these:
list_of_param_lists  = [['PFOA', 'PFOS'], ['PFOA', 'PFOS'], \
                        ['PFNA', 'PFHxS', 'PFBS', 'HFPO-DA'], \
                        ['PFOA', 'PFOS', 'PFNA', 'PFHxS', 'PFBS', 'HFPO-DA']]

In [ ]:
# Loops through and runs once for each of these:
list_of_sample_dicts = [{'DW':'Drinking water', 'GW':'Groundwater'}, \
                        {'SW':'Surface Water', 'WW':'Wastewater'}, \
                        {'DW':'Drinking water', 'GW':'Groundwater', 'SW':'Surface Water', 'WW':'Wastewater'}, \
                        {'Sediment':'Sediment', 'Soil':'Soil'}]

In [ ]:
# Copy STARTING map and add it to the project
for i in range(len(list_of_param_lists)):
    print(i)
    starting_map_name = list_of_start_maps[i]
    param_list = list_of_param_lists[i]
    sample_dict = list_of_sample_dicts[i]
    
    map_to_copy = aprx.listMaps(starting_map_name)[0]
    layout_to_copy = aprx.listLayouts(starting_map_name)[0]
    
    # loop through the parameters and sample types, and generate new maps
    for p in param_list:
        for s in list(sample_dict.keys()):
            newmap_name = f'{prefix}_{p}_{suffix}_{s}'
            
            if starting_map_name != newmap_name:
                # create copies of map and layout with new names
                aprx.copyItem(map_to_copy, newmap_name)
                aprx.copyItem(layout_to_copy, newmap_name)
                newmap = aprx.listMaps(newmap_name)[0]
                # definition query
                for lyr in newmap.listLayers(lyr_to_change):
                    lyr.definitionQuery = f"sample_type = '{sample_dict[s].title()}' and parameter_name = '{p}'"
                # set Layout
                newlayout = aprx.listLayouts(newmap_name)[0]
                mf = newlayout.listElements('mapframe_element')[0]
                mf.map = newmap
                # edit title
                title = newlayout.listElements('text_element', 'Title')[0]
                title.text = f'{p} {sample_dict[s].title()} \n{suffix}'